<a href="https://colab.research.google.com/github/DmitryKutsev/ml_hw/blob/master/hw7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
from google.colab import drive
drive.mount('/content/drive')

!unzip '/content/drive/My Drive/data.zip'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Archive:  /content/drive/My Drive/data.zip
replace avito_data.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace quora.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace __MACOSX/._quora.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [38]:
!pip install pandas scikit-learn gensim matplotlib

In [0]:
import tensorflow as tf

In [0]:
import pandas as pd
import numpy as np
from string import punctuation
from sklearn.model_selection import train_test_split
from collections import Counter
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
avito = pd.read_csv('avito_data.csv').dropna()

In [42]:
avito

,description,deal_probability
0,"Кокон для сна малыша,пользовались меньше месяц...",0.12789
1,"Стойка для одежды, под вешалки. С бутика.",0.00000
2,"В хорошем состоянии, домашний кинотеатр с blu ...",0.43177
3,Продам кресло от0-25кг,0.80323
4,Все вопросы по телефону.,0.20797
...,...,...
1503418,"В хорошем состоянии, ткань плотная (рисунок т...",0.12869
1503420,"Произведем бесплатный замер, изготовим и устан...",0.60000
1503421,"Продам квартиру с отличным ремонтом , две лодж...",0.39569
1503422,Продам красивые туфли в хорошем состоянии!,0.00000


In [0]:
def preprocess(text):
    tokens = text.lower().split()
    tokens = [token.strip(punctuation) for token in tokens]
    return tokens

In [0]:
vocab = Counter()

for text in avito.description:
    vocab.update(preprocess(text))

In [45]:
len(vocab)

1429855

In [0]:
filtered_vocab = set()

for word in vocab:
    if vocab[word] > 30:
        filtered_vocab.add(word)

In [47]:
len(filtered_vocab)

54266

In [0]:
word2id = {'UNK':1, 'PAD':0}

for word in filtered_vocab:
    word2id[word] = len(word2id)

In [0]:
id2word = {i:word for word, i in word2id.items()}

In [0]:
X = []

for text in avito.description:
    tokens = preprocess(text)
    ids = [word2id.get(token, 1) for token in tokens]
    X.append(ids)

In [0]:
MAX_LEN = max(len(x) for x in X)

In [52]:
MAX_LEN

711

In [0]:
X = tf.keras.preprocessing.sequence.pad_sequences(X, maxlen=MAX_LEN)

In [54]:
X.shape

(1387148, 711)

In [0]:
y = avito.deal_probability.values